In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
color = sns.color_palette()
%matplotlib inline
from wordcloud import WordCloud
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import praw
from data import *
import time
import squarify
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import re   # removes links
import en_core_web_sm
import string

In [2]:
#symbols = pd.read_csv('comment_folder/sp500.csv')
#symbols = sp500.values.tolist()
#symbols['Symbol'] = symbols['Symbol'].str[0:4]
#symbols = sp500.values.tolist()
symbols = ["MMM ","AOS ","ABT ","ABBV ","ABMD ","ACN ","ATVI ","ADM ","ADBE ","AAP ","AMD ","AES ","AFL ","APD ","AKAM ","ALK ","ALB ","ARE ","ALGN ","ALLE ","LNT ","GOOGL ","GOOG ",
           "AMZN ","AMC ","AEE ","AAL ","AEP ","AXP ","AIG ","AMT ","AWK ","AMP ","ABC ","AME ","AMGN ","APH ","ADI ","ANSS ","ANTM ","AON ","APA ","AAPL ","AMAT ","APTV ","ANET ","AJG ",
           "AIZ ","ATO ","ADSK ","ADP ","AZO ","AVB ","AVY ","BKR ","BLL ","BAC ","BBWI ","BAX ","BDX ","BRK.B ","BBY ","BIO ","BIIB ","BLK ","BK ","BKNG ","BWA ","BXP ","BSX ","BMY ",
           "AVGO ","BF.B ","CHRW ","COG ","CDNS ","CZR ","CPB ","COF ","CAH ","KMX ","CCL ","CARR ","CTLT ","CAT ","CBOE ","CBRE ","CDW ","CNC ","CNP ","CERN ","CRL ","SCHW ",
           "CHTR ","CVX ","CMG ","CHD ","CINF ","CTAS ","CSCO ","CFG ","CTXS ","CLX ","CME ","CMS ","KO ","CTSH ","CMCSA ","CMA ","CAG ","COP ","STZ ","CPRT ","GLW ",
           "CTVA ","COST ","CCI ","CSX ","CMI ","CVS ","DHI ","DHR ","DRI ","DVA ","DAL ","XRAY ","DVN ","DXCM ","FANG ","DLR ","DFS ","DISCA ","DISCK ","DISH ","DLTR ","DPZ ",
           "DOV ","DOW ","DTE ","DUK ","DRE ","DXC ","EMN ","ETN ","EBAY ","ECL ","EIX ","EW ","LLY ","EMR ","ENPH ","ETR ","EOG ","EFX ","EQIX ","EQR ","ESS ","ETSY ","EVRG ",
           "EXC ","EXPE ","EXPD ","EXR ","XOM ","FFIV ","FB ","FAST ","FRT ","FDX ","FIS ","FITB ","FRC ","FISV ","FLT ","FMC ","FTNT ","FTV ","FBHS ","FOXA ","FOX ","BEN ","FCX ",
           "GPS ","GRMN ","GNRC ","GD ","GIS ","GM ","GPC ","GILD ","GPN ","HAL ","HBI ","HAS ","HCA ","PEAK ","HSIC ","HES ","HPE ","HLT ","HOLX ","HD ","HON ","HRL ","HST ",
           "HWM ","HPQ ","HUM ","HBAN ","HII ","IBM ","IEX ","IDXX ","INFO ","ITW ","ILMN ","INCY ","IR ","INTC ","ICE ","IFF ","IPG ","INTU ","ISRG ","IVZ ","IPGP ","IQV ","IRM ","JBHT ",
           "JKHY ","SJM ","JNJ ","JCI ","JPM ","JNPR ","KSU ","KEY ","KEYS ","KMB ","KIM ","KMI ","KLAC ","KHC ","KR ","LHX ","LH ","LRCX ","LW ","LVS ","LEG ","LDOS ","LEN ","LNC ","LIN ",
           "LYV ","LKQ ","LMT ","LOW ","LUMN ","LYB ","MTB ","MRO ","MPC ","MKTX ","MAR ","MMC ","MLM ","MAS ","MA ","MXIM ","MKC ","MCD ","MCK ","MDT ","MRK ","MET ","MTD ","MGM ","MCHP ",
           "MU ","MSFT ","MAA ","MRNA ","MHK ","TAP ","MDLZ ","MPWR ","MNST ","MCO ","MS ","MSI ","MSCI ","NDAQ ","NTAP ","NFLX ","NWL ","NEM ","NWSA ","NWS ","NEE ","NLSN ","NK ","NSC ",
           "NTRS ","NOC ","NLOK ","NCLH ","NOV ","NRG ","NUE ","NVDA ","NVR ","NXPI ","ORLY ","OXY ","ODFL ","OMC ","OKE ","ORCL ","OGN ","OTIS ","PCAR ","PKG ","PH ","PAYX ","PAYC ","PYPL ",
           "PENN ","PNR ","PBCT ","PEP ","PKI ","PRGO ","PFE ","GE ","PSX ","PNW ","PXD ","PNC ","POOL ","PPG ","PPL ","PFG ","PG ","PGR ","PLD ","PRU ","PTC ","PEG ","PSA ","PHM ","PVH ","QRVO ",
           "QCOM ","PWR ","DGX ","RL ","RJF ","RTX ","REG ","REGN ","RF ","RSG ","RMD ","RHI ","ROK ","ROL ","ROP ","ROST ","RCL ","SPGI ","CRM ","SBAC ","SLB ","STX ","SEE ","SRE ","NOW ",
           "SHW ","SPG ","SWKS ","SNA ","LUV ","SWK ","SBUX ","STT ","STE ","SYK ","SIVB ","SYF ","SNPS ","SYY ","TMUS ","TROW ","TTWO ","TPR ","TGT ","TEL ","TDY ","TFX ","TER ","TSLA ",
           "TXN ","TXT ","COO ","HIG ","HSY ","MOS ","TRV ","DIS ","TMO ","TJX ","TSCO ","TT ","TDG ","TRMB ","TFC ","TWTR ","TYL ","TSN ","USB ","UDR ","ULTA ","UAA ","UA ","UNP ","UAL ","UPS ",
           "URI ","UNH ","UHS ","UNM ","VLO ","VTR ","VRSN ","VRSK ","VZ ","VRTX ","VFC ","VIAC ","VTRS ","VNO ","VMC ","WRB ","GWW ","WAB ","WBA ","WMT ","WM ","WAT ","WEC ","WFC ","WELL ",
           "WST ","WDC ","WU ","WRK ","WY ","WHR ","WMB ","WLTW ","WYNN ","XEL ","XLNX ","XYL ","YUM ","ZBRA ","ZBH ","ZION ","ZTS ", "GME ", 'BB ']

In [14]:
df = pd.read_csv('comment_folder/all_comments.csv')
#df = pd.read_csv('comment_folder/all_comments.csv')
# df = df(["Score"] > 10)
#df = df[df['Score'] > 10]

df.head()

,Unnamed: 0,Author,Comment,Score,timestamp
0,249,KarlMarx693,Is Jpow the most memed financial official ever?,13,6/16/2021 5:06
1,193,pr3dato8,Is it too late to get into $JPOW?,14,6/16/2021 5:07
2,404,dragonrider85,"Gentlemen, it is with great pleasure to inform...",36,6/16/2021 5:09
3,403,MLKKO,Whos still holding BB?,152,6/16/2021 5:11
4,127,None,Last week I went to go outside for a smoke and...,98,6/16/2021 5:15


In [15]:
search_list = symbols

df['symbols'] = df.Comment.str.extract('({0})'.format('|'.join(search_list)))
symbols_df = df[~pd.isna(df.symbols)]

symbols_df.tail()

,Unnamed: 0,Author,Comment,Score,timestamp,symbols
28301,216,Hans-Diamond,"""Please explain your path to Fed Chair""\n\n \...",10,8/17/2021 13:07,ARE
28354,314,Pifoot_trading,AAPL has declared war on all guys with small d...,8,8/17/2021 13:46,AAPL
28355,314,Pifoot_trading,AAPL has declared war on all guys with small d...,10,8/17/2021 13:46,AAPL
28402,101,prisonsexx,PFE calls saving my portfolio,13,8/17/2021 14:17,PFE
28404,101,prisonsexx,PFE calls saving my portfolio,13,8/17/2021 14:17,PFE


In [16]:
#symbols_df = symbols_df[(symbols_df['timestamp'] > '2021-08-16 00:30:00') & (symbols_df['timestamp'] < '2021-08-17 23:00:00')]
#symbols_df

In [17]:
top_tickers = symbols_df['symbols'].value_counts().head(15)
top_tickers

TSLA     310
BB       207
AAPL     175
MRNA     115
NVDA      96
AMD       94
DTE       76
MU        75
ARE       55
AMZN      55
PFE       52
AMC       48
MSFT      41
GME       38
DIS       34
Name: symbols, dtype: int64

In [18]:
# Create stopword list:
stopwords = set(stopwords.words('english'))
textt = " ".join(comment for comment in symbols_df.Comment)
wordcloud = WordCloud(stopwords=stopwords).generate(textt)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()


AttributeError: 'set' object has no attribute 'words'

In [ ]:
# assign reviews with score > 3 as positive sentiment
# score < 3 negative sentiment
# remove score = 3
#symbols_df = symbols_df[symbols_df['sentiment'] != 0]
#symbols_df['sentiment'] = symbols_df['Score'].apply(lambda rating : +1 if rating > 0 else -1)

In [20]:
# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.01:  # Positive
        result = 1
    elif score <= -0.01:  # Negative
        result = -1

    return result

In [21]:
analyzer = SentimentIntensityAnalyzer()

In [22]:
# Sentiment scores dictionaries
text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "sentiment": [],
}

# Get sentiment for the text and the title
for index, row in symbols_df.iterrows():
    try:
        # Sentiment scoring with VADER
        
        text_sentiment = analyzer.polarity_scores(row["Comment"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["sentiment"].append(get_sentiment(text_sentiment["compound"]))
    
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
text_sentiment_df = pd.DataFrame(text_sent)
symbols_df = symbols_df.join(text_sentiment_df)

In [23]:
symbols_df.head()

,Unnamed: 0,Author,Comment,Score,timestamp,symbols,text_compound,text_pos,text_neu,text_neg,sentiment
16,10,RADIO02118,"Remember when TSLA would go up 10% a week, eve...",51,6/16/2021 6:06,TSLA,0.0000,0.000,1.000,0.0,0.0
23,246,ktrash,Yellen going to steal the mic at FOMC today an...,43,6/16/2021 6:30,OMC,0.6940,0.656,0.344,0.0,1.0
47,254,ImHereForLaughs58,BB may be a long plan now,14,6/16/2021 7:30,BB,0.7430,0.310,0.690,0.0,1.0
66,321,Arizonaj9,WISH CLF KMI go,13,6/16/2021 8:12,KMI,0.0000,0.000,1.000,0.0,0.0
69,413,JayArlington,2 Steel companies (NUE and STLD) just revised ...,11,6/16/2021 8:14,NUE,0.6908,0.655,0.345,0.0,1.0


In [24]:
dfNew = symbols_df[['Comment',"symbols",'sentiment']]
dfNew = dfNew.dropna()
dfNew

,Comment,symbols,sentiment
16,"Remember when TSLA would go up 10% a week, eve...",TSLA,0.0
23,Yellen going to steal the mic at FOMC today an...,OMC,1.0
47,BB may be a long plan now,BB,1.0
66,WISH CLF KMI go,KMI,0.0
69,2 Steel companies (NUE and STLD) just revised ...,NUE,1.0
...,...,...,...
1667,I need power hour to send NVDA to $780,NVDA,1.0
1684,Got fucked by 0tde NVDA calls...I guess no mor...,NVDA,0.0
1685,Got fucked by 0tde NVDA calls...I guess no mor...,NVDA,0.0
1724,I need to load up on more TSLA calls. $45k isn...,TSLA,0.0


In [25]:
dff = dfNew.groupby(["symbols"]).sentiment.sum().reset_index()
dff = dff.sort_values(by='sentiment', ascending=False)
dff

,symbols,sentiment
14,NVDA,10.0
0,AAPL,2.0
2,AMD,2.0
16,OMC,2.0
7,GME,2.0
1,AMC,1.0
21,VIAC,1.0
17,SEE,1.0
13,NUE,1.0
22,XOM,1.0


In [16]:
# random split train and test data
index = dfNew.index
dfNew['random_number'] = np.random.randn(len(index))
train = dfNew[dfNew['random_number'] <= 0.8]
test = dfNew[dfNew['random_number'] > 0.8]

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Comment'])
test_matrix = vectorizer.transform(test['Comment'])

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']

In [ ]:
lr.fit(X_train,y_train)


In [ ]:
predictions = lr.predict(X_test)

In [ ]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)


In [ ]:
print(classification_report(predictions,y_test))